# Demo: Spatial reasoning using WiFi Fingerprinting

Course: NICF- Spatial Reasoning from Sensor Data (SF)

Website: https://www.iss.nus.edu.sg/executive-education/course/detail/spatial-reasoning-from-sensor-data/artificial-intelligence

Contact: Dr. Tian Jing

Email: tianjing@nus.edu.sg

- Dataset: UJIIndoorLoc Dataset: https://www.kaggle.com/giantuji/UjiIndoorLoc

In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
import os
from sklearn.preprocessing import scale
from keras.models import Sequential
from keras.layers import Dense, Dropout

Using TensorFlow backend.


In [2]:
# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # model will be trained on GPU

In [3]:
path_train = "UJIIndoorLoc_trainingData.csv"
path_validation = "UJIIndoorLoc_validationData.csv"

#Explicitly pass header=0 to be able to replace existing names 
train_df = pd.read_csv(path_train,header = 0)
train_df = train_df[:19930]
train_AP_strengths = train_df.iloc[:,:520] #select first 520 columns

#Scale transforms data to center to the mean and component wise scale to unit variance
train_AP_features = scale(np.asarray(train_AP_strengths).astype('float64'))

#The following two objects are actually pandas.core.series.Series objects
building_ids_str = train_df["BUILDINGID"].map(str) #convert all the building ids to strings
building_floors_str = train_df["FLOOR"].map(str) #convert all the building floors to strings

res = building_ids_str + building_floors_str #element wise concatenation of BUILDINGID+FLOOR
train_labels = np.asarray(building_ids_str + building_floors_str)

#convert labels to categorical variables, dummy_labels has type 'pandas.core.frame.DataFrame'
dummy_labels = pd.get_dummies(train_labels)

# one hot encode the dummy_labels. this is done because dummy_labels is a dataframe with the labels (BUILDINGID+FLOOR) as the column names

train_labels = np.asarray(dummy_labels)

print(train_labels.shape)

(19930, 13)


In [4]:
# Split dataset
train_val_split = np.random.rand(len(train_AP_features))
train_val_split = train_val_split < 0.70 #should contain ~70% percent true
# We will then split our given training set into training + validation 
train_X = train_AP_features[train_val_split]
train_y = train_labels[train_val_split]
val_X = train_AP_features[~train_val_split]
val_y = train_labels[~train_val_split]


In [5]:
#Turn the given validation set into a testing set
test_df = pd.read_csv(path_validation,header = 0)
test_AP_features = scale(np.asarray(test_df.iloc[:,0:520]))
test_labels = np.asarray(test_df["BUILDINGID"].map(str) + test_df["FLOOR"].map(str))
test_labels = np.asarray(pd.get_dummies(test_labels))

C:\Users\Tian\Anaconda3\envs\srsdv\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


In [6]:
nb_epochs = 10
batch_size = 10
input_size = 520
num_classes = 13

In [7]:
#Define model
def encoder():
    model = Sequential()
    model.add(Dense(256, input_dim=input_size, activation='tanh', use_bias=True))
    model.add(Dense(128, activation='tanh', use_bias=True))
    model.add(Dense(64, activation='tanh', use_bias=True))
    return model

def decoder(e):   
    e.add(Dense(128, input_dim=64, activation='tanh', use_bias=True))
    e.add(Dense(256, activation='tanh', use_bias=True))
    e.add(Dense(input_size, activation='tanh', use_bias=True))
    e.compile(optimizer='adam', loss='mse')
    return e
    
e = encoder()
e.summary()

d_pre = decoder(e)
d_pre.summary()
 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 256)               133376    
_________________________________________________________________
dense_2 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_3 (Dense)              (None, 64)                8256      
Total params: 174,528
Trainable params: 174,528
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 256)               133376    
_________________________________________________________________
dense_2 (Dense)              (None, 128)               32896     
_________________________________________________________________
dens

In [8]:
d_pre.fit(train_X, train_X, epochs=nb_epochs, batch_size=batch_size)

Epoch 1/10
13935/13935 [==============================] - 19s 1ms/step - loss: 0.6963
Epoch 2/10
13935/13935 [==============================] - 16s 1ms/step - loss: 0.6724: 0s - loss: 0.671
Epoch 3/10
13935/13935 [==============================] - 20s 1ms/step - loss: 0.6667
Epoch 4/10
13935/13935 [==============================] - 15s 1ms/step - loss: 0.6637
Epoch 5/10
13935/13935 [==============================] - 15s 1ms/step - loss: 0.6617
Epoch 6/10
13935/13935 [==============================] - 16s 1ms/step - loss: 0.6598
Epoch 7/10
13935/13935 [==============================] - 15s 1ms/step - loss: 0.6582
Epoch 8/10
13935/13935 [==============================] - 15s 1ms/step - loss: 0.6569
Epoch 9/10
13935/13935 [==============================] - 16s 1ms/step - loss: 0.6557: 0s
Epoch 10/10
13935/13935 [==============================] - 15s 1ms/step - loss: 0.6544


In [9]:
# Based on trained autoencoder features, define a classification model

def classifier(d):
    num_to_remove = 3
    for i in range(num_to_remove):
        d.pop()
    d.add(Dense(128, input_dim=64, activation='tanh', use_bias=True))
    d.add(Dense(128, activation='tanh', use_bias=True))
    d.add(Dense(num_classes, activation='softmax', use_bias=True))
    d.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return d

c = classifier(d_pre)
c.fit(train_X, train_y, validation_data=(val_X, val_y), epochs=nb_epochs, batch_size=batch_size)
c.summary()


Train on 13935 samples, validate on 5995 samples
Epoch 1/10
13935/13935 [==============================] - 18s 1ms/step - loss: 0.3304 - acc: 0.8841 - val_loss: 0.1917 - val_acc: 0.9403
Epoch 2/10
13935/13935 [==============================] - 12s 858us/step - loss: 0.1434 - acc: 0.9532 - val_loss: 0.1702 - val_acc: 0.9466
Epoch 3/10
13935/13935 [==============================] - 11s 759us/step - loss: 0.1160 - acc: 0.9616 - val_loss: 0.1651 - val_acc: 0.9498
Epoch 4/10
13935/13935 [==============================] - 10s 703us/step - loss: 0.0990 - acc: 0.9680 - val_loss: 0.1571 - val_acc: 0.9578
Epoch 5/10
13935/13935 [==============================] - 10s 686us/step - loss: 0.0926 - acc: 0.9672 - val_loss: 0.1499 - val_acc: 0.9551
Epoch 6/10
13935/13935 [==============================] - 10s 744us/step - loss: 0.0828 - acc: 0.9734 - val_loss: 0.1474 - val_acc: 0.9545
Epoch 7/10
13935/13935 [==============================] - 12s 836us/step - loss: 0.0690 - acc: 0.9760 - val_loss: 0.156

In [10]:
# Evaluate the performance

loss, acc = c.evaluate(test_AP_features, test_labels, verbose=0)
print('Accuracy: %.4f' % acc)

Accuracy: 0.7066


Have a nice day!